In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/goodyear-pdf'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/goodyear-pdf/05-08-2017 Anti-Bribery Policy FINAL Links updated 07-14-21.pdf
/kaggle/input/goodyear-pdf/Policy on Policy Governance (final).pdf
/kaggle/input/goodyear-pdf/Global Travel Expense Policy English.pdf
/kaggle/input/goodyear-pdf/Corporate_Fraud_Policy.pdf
/kaggle/input/goodyear-pdf/Finance Master Data Playbook.pdf


In [8]:
import sys
sys.version

'3.10.10 | packaged by conda-forge | (main, Mar 24 2023, 20:08:06) [GCC 11.3.0]'

In [9]:
!pip install streamlit
!pip install streamlit-chat
!pip install langchain tiktoken chromadb pypdf transformers InstructorEmbedding
!pip install accelerate bitsandbytes sentencepiece Xformers

In [10]:
import torch
import transformers
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("lmsys/fastchat-t5-3b-v1.0")
model = AutoModelForSeq2SeqLM.from_pretrained("lmsys/fastchat-t5-3b-v1.0")

In [11]:
# import torch
# import transformers
# from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, Pipeline

# import transformers
import langchain
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

pipe = pipeline(
                "text2text-generation",
                model=model,
                tokenizer=tokenizer,
                max_length=256)

local_llm = HuggingFacePipeline(pipeline=pipe) # got an erro says 'return_full_text' is not used

In [12]:
# input texts
prompt = "what is the capital of the United States?"
# encoding the input texts
input_ids = tokenizer.encode(prompt, return_tensors="pt") # 'pt' is pytorch tensor
# set parameters
max_length = 256  # max length of the generated text
num_beams = 4  # width of the beam search
num_return_sequences = 1  # num of the most possible returned sequence
# use model to generate texts
outputs = model.generate(input_ids, max_length=max_length, num_beams=num_beams, num_return_sequences=num_return_sequences)  
# decode the id to texts
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

<pad> washington dc


In [13]:
import os
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

# from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

In [14]:
# Load and process the text files
# loader = TextLoader("single_text_file.txt")
loader = DirectoryLoader("/kaggle/input/goodyear-pdf", glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

In [15]:
len(documents) # Total pages of all the PDF files

115

In [16]:
print(documents[1])

page_content='Replaces Policy Dated July 1, 2011 \n2 \n Anti -Bribery  Policy  \nIntroduction  \n \nAs part of our worldwide commitment to honesty, integrity and respect, Goodyear does not wish to \nobtain business advantages by offering or receiving  improper payments  or things  of value , even in \ncountries where such practices may be socially and culturally accepted . The United States  (U.S.) \nForeign Corrupt Practices Act  (FCPA ), the OECD Convention on Combating Bribery of Foreign \nPublic Officials in International Business Transactions , the UK B ribery Act  and the United Nations \nConvention Against Corruption, as well as numerous other anticorruption laws around the world, \nundersc ore the worldwide concern over bribery.  \n \nGoodyear’s Anti -Bribery P olicy (“Policy”) is that no Goodyear associate will , directly or \nindirectly,  offer , pay, promise to pay, authorize the payment of, receive or accept , any improper \npayment or anything of value  to or from  anyone 

In [17]:
# Splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=10)
texts = text_splitter.split_documents(documents)

In [18]:
len(texts)

1343

In [19]:
texts[2]

Document(page_content='obtain business advantages by offering or receiving  improper payments  or things  of value , even in \ncountries where such practices may be socially and culturally accepted . The United States  (U.S.)', metadata={'source': '/kaggle/input/goodyear-pdf/05-08-2017 Anti-Bribery Policy FINAL Links updated 07-14-21.pdf', 'page': 1})

In [20]:
!pip install InstructorEmbedding sentence_transformers huggingface-hub

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 54.4 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.4/346.4 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 56.4 MB/s eta 0:00:00:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.4.0
    Uninstalling fsspec-2023.4.0:
      Successfully uninstalled fsspec-2023.4.0
  Attempting 

In [25]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl", model_kwargs={"device":"cpu"})

ImportError: Dependencies for InstructorEmbedding not found.

In [19]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
# !pip install chromadb
persist_directory = "goodyear_output"

# Here is the new embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts, embedding=embedding, persist_directory=persist_directory)

In [27]:
# persiste the db to disk
vectordb.persist()
# vectordb = None

In [30]:
# Now we can load the persisted database from the disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

retriever = vectordb.as_retriever()

docs = retriever.get_relevant_documents("What are the responsibilities of a traveler?")

In [36]:
len(docs)

4

In [37]:
docs[1]

Document(page_content="page 6 OVERSEER  \nA.  Goodyear’s VP and Controller  is responsible for the development, distribu tion, and \ntraining of the global policy and the country -specific policy addendums, if applicable.  \nB. The Global Travel & Expense Policy Governance Committee  is responsible for reviewing \nand approving all changes to this policy and country -specific policy a ddendums, if \napplicable.  \n \nC. The Global Travel & Expense Organization  defines the process to monitor tax, legal and \npolicy compliance.  \n \nD. Internal Audit department and/or Travel and Expense departments  will conduct audits \nof expense reports and investigate instances of non -compliance with this policy.  In \naddition, these functions will monitor Corporate Card usage and payment requirements.  \n \nF. Manager and Human Resources  must notify the proper administrato r to ensure an \nassociate’s corporate credit card is cancelled when:  \no it is no longer required for the associate's dut

In [32]:
# Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split("\n")
    
    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    
    # Join the wrapped lines back together using newline characters
    wrapped_text = "\n".join(wrapped_lines)
    
    return wrapped_text

def process_llm_response(llm_response):
    # print(wrap_text_preserve_newlines(llm_response['result']))
    # print("\n\nSources:")
    data = {}
    dh = []
    for source in llm_response['source_documents']:
        dh.append(source.metadata['source'])
    data['Source'] = dh
    
    return wrap_text_preserve_newlines(llm_response['result']), data

In [33]:
# Create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=local_llm,
                                      chain_type='stuff',
                                      retriever=retriever,
                                      return_source_documents=True)

In [34]:
query = "What is the definition of Fraud ?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Token indices sequence length is longer than the specified maximum sequence length for this model (2112 > 2048). Running this sequence through the model will result in indexing errors


('<pad>  The  definition  of  fraud  is  the  intentional,  false  representation  or  concealment  of  facts\nfor  the  purpose  of  inducing  another  to  act  upon  such  representations  or  concealment  to  his  or\nher  injury,  a  deception  intended  to  result  in  financial  or  personal  gain,  and/or  the  intentional\nmisallocation,  misappropriati  on,  and/or  improper  conversion  of  Company-owned  assets.\n',
 {'Source': ['/kaggle/input/goodyear-pdf/Corporate_Fraud_Policy.pdf',
   '/kaggle/input/goodyear-pdf/Corporate_Fraud_Policy.pdf',
   '/kaggle/input/goodyear-pdf/Corporate_Fraud_Policy.pdf',
   '/kaggle/input/goodyear-pdf/Corporate_Fraud_Policy.pdf']})

In [35]:
query = "What is the “Improper payments” or “things of value”?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

('<pad>  Any  payment  or  thing  of  value  that  is  known  or  suspected  to  be  provided  to  a  non-USA\ngovernment  official  or  other  person  in  order  to  obtain  or  retain  business  or  to  secure  any\nimproper  advantage  for  Goodyear.\n',
 {'Source': ['/kaggle/input/goodyear-pdf/05-08-2017 Anti-Bribery Policy FINAL Links updated 07-14-21.pdf',
   '/kaggle/input/goodyear-pdf/05-08-2017 Anti-Bribery Policy FINAL Links updated 07-14-21.pdf',
   '/kaggle/input/goodyear-pdf/05-08-2017 Anti-Bribery Policy FINAL Links updated 07-14-21.pdf',
   '/kaggle/input/goodyear-pdf/Corporate_Fraud_Policy.pdf']})